## 锚框

目标检测算法通常会在输入图像中采样大量的区域，然后判断这些区域中是否包含我们感兴趣的目标，并调整区域边界从而更准确地预测目标的真实边界框（ground-truth bounding box）。

其中一种区域采样方法，以每个像素为中心，生成多个缩放比和宽高比（aspect ratio）不同的边界框。这些边界框被称为锚框（anchor box）。

一类目标检测算法是基于锚框的：
- 提出多个锚框；
- 预测每个锚框里，是否含有相关注的物体；
- 如果是，则预测从这个锚框到真实边缘框的偏移；

In [1]:
%matplotlib inline
import torch
import sys
sys.path.append('..')
import utils
import d2l

torch.set_printoptions(2)  # 精简输出精度

### 1、生成多个锚框

算法固定生成，或者根据图片来生成；

输入图像高度是$h$，宽度是$w$，以图像的每个像素为中心，生成不同形状的锚框：
- 中心位置、缩放比、宽高比，唯一确定一个锚框；
- 缩放比（scale）$s\in(0, 1]$，宽高比（aspect ratio）为$r>0$；
- 则锚框的宽度为$ws\sqrt{r}$，高度为$\frac{hs}{\sqrt{r}}$；

要生成许多不同形状的锚框，需要预先设置的缩放比取值$s_1,...,s_n$和许多宽高比取值$r_1,...,r_m$：
- 排列组合，则共$whmn$个锚框，太多；
- 实践中，只考虑包含$s_1$或$r_1$的组合：$(s_1,r_1),...,(s_1,r_m),(s_2,r_1),...,(s_n,r_1)$；共$wh(n+m-1)$个。

In [ ]:
def multibox_prior(data, sizes, ratios):
    """
    指定输入图像、尺寸列表和宽高比列表，生成以每个像素为中心具有不同形状的锚框
    
    Parameters:
    - data: 输入图像
    - sizes: 尺寸列表
    - ratios: 宽高比列表
    """
    in_height, in_width = data.shape[-2:]
    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    boxes_per_pixel = (num_sizes + num_ratios - 1)
    size_tensor = torch.tensor(sizes, device=device)
    ratio_tensor = torch.tensor(ratios, device=device)

    # 为了将锚点移动到像素的中心，需要设置偏移量。
    # 因为一个像素的的高为1且宽为1，我们选择偏移我们的中心0.5
    offset_h, offset_w = 0.5, 0.5
    steps_h = 1.0 / in_height  # 在y轴上缩放步长，为了归一化到(0, 1]，方便后面的锚框高宽表示（不需要再乘图像的高宽了）
    steps_w = 1.0 / in_width  # 在x轴上缩放步长

    # 生成锚框的所有中心点
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    shift_y, shift_x = torch.meshgrid(center_h, center_w)
    shift_y, shift_x = shift_y.reshape(-1), shift_x.reshape(-1)

    # 生成“boxes_per_pixel”个高和宽，
    # 之后用于创建锚框的四角坐标(xmin,xmax,ymin,ymax)
    w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]),
                   sizes[0] * torch.sqrt(ratio_tensor[1:])))\
                   * in_height / in_width  # 处理矩形输入
    h = torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]),
                   sizes[0] / torch.sqrt(ratio_tensor[1:])))
    # 除以2来获得半高和半宽
    anchor_manipulations = torch.stack((-w, -h, w, h)).T.repeat(
                                        in_height * in_width, 1) / 2

    # 每个中心点都将有“boxes_per_pixel”个锚框，
    # 所以生成含所有锚框中心的网格，重复了“boxes_per_pixel”次
    out_grid = torch.stack([shift_x, shift_y, shift_x, shift_y],
                dim=1).repeat_interleave(boxes_per_pixel, dim=0)
    output = out_grid + anchor_manipulations
    return output.unsqueeze(0)

In [ ]:
#@save
def show_bboxes(axes, bboxes, labels=None, colors=None):
    """显示所有边界框"""
    def _make_list(obj, default_values=None):
        if obj is None:
            obj = default_values
        elif not isinstance(obj, (list, tuple)):
            obj = [obj]
        return obj

    labels = _make_list(labels)
    colors = _make_list(colors, ['b', 'g', 'r', 'm', 'c'])
    for i, bbox in enumerate(bboxes):
        color = colors[i % len(colors)]
        rect = utils.bbox_corner_to_rect(bbox.detach().numpy(), color)
        axes.add_patch(rect)
        if labels and len(labels) > i:
            text_color = 'k' if color == 'w' else 'w'
            axes.text(rect.xy[0], rect.xy[1], labels[i],
                      va='center', ha='center', fontsize=9, color=text_color,
                      bbox=dict(facecolor=color, lw=0))

### 2、IoU - 交并比

- IoU（Intersection over Union）用于计算两个框之间的相似度
  - 0对应无重叠，1对应重合；
- 这是Jacquard杰卡德相似度的一个特殊情况：
  - 给定两个集合A和B，$J(A,B) = \frac{|A \cap B|}{|A \cup B|}$
  - IoU为两个边界框，相交面积与相并面积之比

### 3、在训练数据中标注锚框

赋予锚框标号：
- 每个锚框是一个训练样本；边界框指的是ground truth；
- 将每个锚框，要么标注成背景，要么关联上一个最接近此锚框的真实边缘框；
- 算法可能会生成大量的锚框：
  - 会导致大量的负样本；

为了训练目标检测模型，我们需要每个锚框的类别（class）和偏移量（offset）标签，其中前者是与锚框相关的对象的类别，后者是真实边界框相对于锚框的偏移量。 在预测时，我们为每个图像生成多个锚框，预测所有锚框的类别和偏移量，根据预测的偏移量调整它们的位置以获得预测的边界框，最后只输出符合特定条件的预测边界框。

#### 3.1、将最接近的真实边缘框分配给锚框

<img src='./figs/1.jpg' style="zoom:20%;" />

- 每个锚框与每个边缘框算IoU值，生成锚框-边缘框的2d矩阵；
- 每次取最大值并删除最大值所在行列，直到所有的边缘框都对应到一个锚框；此步中生成的全是正例；
- 剩余的锚框（即剩余训练样本）可：
  - 全部标注成“背景”，即全是负例，会导致大量负样本；
  - 寻找与本锚框IoU最大的边界框，若值大于阈值（ *能不能做动态阈值？* ），则将该边界框assign给本锚框。

#### 3.2、标注类别和偏移量

### 4、使用非极大值抑制（NMS）来预测边界框

在预测时，我们先为图像生成多个锚框，再为这些锚框一一预测类别和偏移量。

当有许多锚框时，可能会输出许多相似的具有明显重叠的锚框，都围绕着同一目标。 为了简化输出，我们可以使用非极大值抑制（non-maximum suppression，NMS）合并属于同一目标的类似的锚框。

在同一张图像中，所有预测的非背景锚框都按置信度降序排序，以生成列表 L 。然后我们通过以下步骤操作排序列表 L ：
- 选中 L 中非基准的、最大置信度的锚框B，加入基准；
- 去除所有其它和B的IoU值大于$\epsilon$的非基准锚框；
- 重复上述过程，直到 L 中都为基准；此时 L 中所锚框两两之间的IoU小于$\epsilon$。

### 5、小结

- 一类目标检测算法基于锚框来预测；
- 首先生成大量锚框，并赋予标号，每个锚框作为一个样本进行训练；
- 在预测时，使用NMS来去掉冗余的预测。